# Entraînement du modèle Random Forest pour la détection de pneumonie

Ce notebook implémente l'entraînement d'un modèle Random Forest pour détecter la pneumonie à partir de radiographies pulmonaires. Nous utiliserons les images prétraitées du notebook précédent.

Les étapes principales sont:
1. Chargement des images prétraitées
2. Extraction des caractéristiques des images
3. Entraînement du modèle Random Forest 
4. Évaluation des performances du modèle
5. Optimisation des hyperparamètres

**os:** pour naviguer dans les dossiers d'images

**numpy:** pour manipuler les tableaux de données

**pandas:** pour gérer les dataframes

**matplotlib:** pour visualiser les résultats

**PIL:** pour charger les images

**sklearn:** pour implémenter le modèle Random Forest et évaluer ses performances

In [24]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import time
import pickle 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import GridSearchCV

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# La cellule ci-dessous vérifie que nous pouvons accéder aux images prétraitées et affiche leur répartition.

In [25]:
preprocessed_base = 'preprocessed_chest_xray'
train_dir = os.path.join(preprocessed_base, 'train')
val_dir = os.path.join(preprocessed_base, 'val')
test_dir = os.path.join(preprocessed_base, 'test')

for directory in [train_dir, val_dir, test_dir]:
    if not os.path.exists(directory):
        print(f"Attention: Le dossier {directory} n'existe pas!")
    else:
        normal_count = len(os.listdir(os.path.join(directory, 'NORMAL')))
        pneumonia_count = len(os.listdir(os.path.join(directory, 'PNEUMONIA')))
        print(f"{directory}: {normal_count} images normales, {pneumonia_count} images pneumonie")

preprocessed_chest_xray/train: 3418 images normales, 3418 images pneumonie
preprocessed_chest_xray/val: 427 images normales, 427 images pneumonie
preprocessed_chest_xray/test: 428 images normales, 428 images pneumonie


# Fonction d'extraction des charactéristiques de l'image

In [26]:
def load_and_extract_features(directory, max_samples=None):
    """
    Charge les images depuis un répertoire et extrait des caractéristiques basiques.
    
    Paramètres:
    -----------
    directory : str
        Chemin vers le répertoire contenant les sous-dossiers 'NORMAL' et 'PNEUMONIA'
    max_samples : int ou None
        Nombre maximum d'échantillons à charger par classe (None pour tout charger)
        
    Retours:
    --------
    features : numpy.ndarray
        Tableau des caractéristiques extraites (statistiques basiques)
    labels : numpy.ndarray
        Tableau des étiquettes (0 pour normal, 1 pour pneumonie)
    """
    features = []
    labels = []
    
    for class_label, class_name in enumerate(['NORMAL', 'PNEUMONIA']):
        class_dir = os.path.join(directory, class_name)
        

        image_files = [f for f in os.listdir(class_dir) 
                      if f.lower().endswith(('.jpeg', '.jpg', '.png'))]
        

        if max_samples is not None:
            image_files = image_files[:max_samples]
        
        print(f"Chargement de {len(image_files)} images {class_name}...")
        

        for i, img_file in enumerate(image_files):
    
            if i > 0 and i % 100 == 0:
                print(f"  - {i}/{len(image_files)} images traitées...")
            
    
            img_path = os.path.join(class_dir, img_file)
            img = Image.open(img_path)
            img_array = np.array(img)
            
    
    
            mean_val = np.mean(img_array)
            std_val = np.std(img_array)  
            min_val = np.min(img_array)  
            max_val = np.max(img_array)  
            median_val = np.median(img_array)
            
    
            h, w = img_array.shape
            h_mid, w_mid = h // 2, w // 2
            
    
            q1 = img_array[:h_mid, :w_mid]
            q1_mean = np.mean(q1)
            q1_std = np.std(q1)
            
    
            q2 = img_array[:h_mid, w_mid:]
            q2_mean = np.mean(q2)
            q2_std = np.std(q2)
            
    
            q3 = img_array[h_mid:, :w_mid]
            q3_mean = np.mean(q3)
            q3_std = np.std(q3)
            
    
            q4 = img_array[h_mid:, w_mid:]
            q4_mean = np.mean(q4)
            q4_std = np.std(q4)
            
    
            img_features = [
                mean_val, std_val, min_val, max_val, median_val,
                q1_mean, q1_std, q2_mean, q2_std,
                q3_mean, q3_std, q4_mean, q4_std
            ]
            
    
            features.append(img_features)
            labels.append(class_label)
    
    return np.array(features), np.array(labels)

In [27]:
print("Extraction des caractéristiques de l'ensemble d'entraînement...")
train_features, train_labels = load_and_extract_features(train_dir)

print("\nExtraction des caractéristiques de l'ensemble de validation...")
val_features, val_labels = load_and_extract_features(val_dir)

print("\nDimensions des données:")
print(f"Caractéristiques d'entraînement: {train_features.shape}")
print(f"Étiquettes d'entraînement: {train_labels.shape}")
print(f"Caractéristiques de validation: {val_features.shape}")
print(f"Étiquettes de validation: {val_labels.shape}")

Extraction des caractéristiques de l'ensemble d'entraînement...
Chargement de 3418 images NORMAL...
  - 100/3418 images traitées...
  - 200/3418 images traitées...
  - 300/3418 images traitées...
  - 400/3418 images traitées...
  - 500/3418 images traitées...
  - 600/3418 images traitées...
  - 700/3418 images traitées...
  - 800/3418 images traitées...
  - 900/3418 images traitées...
  - 1000/3418 images traitées...
  - 1100/3418 images traitées...
  - 1200/3418 images traitées...
  - 1300/3418 images traitées...
  - 1400/3418 images traitées...
  - 1500/3418 images traitées...
  - 1600/3418 images traitées...
  - 1700/3418 images traitées...
  - 1800/3418 images traitées...
  - 1900/3418 images traitées...
  - 2000/3418 images traitées...
  - 2100/3418 images traitées...
  - 2200/3418 images traitées...
  - 2300/3418 images traitées...
  - 2400/3418 images traitées...
  - 2500/3418 images traitées...
  - 2600/3418 images traitées...
  - 2700/3418 images traitées...
  - 2800/3418 imag

# On feed la data à Random Forest qui va s'entraîner et générer des conclusions, et on sauvegarde les résultats

In [ ]:
import os


numpy_models_dir = 'numpy_models'
if not os.path.exists(numpy_models_dir):
    os.makedirs(numpy_models_dir)
    print(f"Dossier '{numpy_models_dir}' créé avec succès.")
else:
    print(f"Le dossier '{numpy_models_dir}' existe déjà.")

print("Entraînement du modèle Random Forest...")
start_time = time.time()

rf_model = RandomForestClassifier(
    n_estimators=100, 
    max_depth=None,   
    min_samples_split=2, 
    random_state=42   
)

rf_model.fit(train_features, train_labels)

training_time = time.time() - start_time
print(f"Modèle entraîné en {training_time:.2f} secondes")

val_predictions = rf_model.predict(val_features)

accuracy = accuracy_score(val_labels, val_predictions)
precision = precision_score(val_labels, val_predictions)
recall = recall_score(val_labels, val_predictions)
f1 = f1_score(val_labels, val_predictions)

print("\nPerformances sur l'ensemble de validation:")
print(f"Précision (accuracy): {accuracy:.4f}")
print(f"Précision (precision): {precision:.4f}")
print(f"Rappel (recall): {recall:.4f}")
print(f"Score F1: {f1:.4f}")

conf_matrix = confusion_matrix(val_labels, val_predictions)
print("\nMatrice de confusion:")
print(conf_matrix)

print("\nRapport de classification:")
print(classification_report(val_labels, val_predictions, 
                          target_names=['Normal', 'Pneumonie']))


with open(os.path.join(numpy_models_dir, 'rf_model.pkl'), 'wb') as f:
    pickle.dump(rf_model, f)
with open(os.path.join(numpy_models_dir, 'train_features.pkl'), 'wb') as f:
    pickle.dump(train_features, f)
with open(os.path.join(numpy_models_dir, 'train_labels.pkl'), 'wb') as f:
    pickle.dump(train_labels, f)
with open(os.path.join(numpy_models_dir, 'val_features.pkl'), 'wb') as f:
    pickle.dump(val_features, f)
with open(os.path.join(numpy_models_dir, 'val_labels.pkl'), 'wb') as f:
    pickle.dump(val_labels, f)

print(f"Modèle et caractéristiques sauvegardés avec succès dans le dossier '{numpy_models_dir}'!")


metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'confusion_matrix': conf_matrix.tolist()
}

with open(os.path.join(numpy_models_dir, 'performance_metrics.pkl'), 'wb') as f:
    pickle.dump(metrics, f)
print(f"Métriques de performance sauvegardées dans le dossier '{numpy_models_dir}'!")

Dossier 'numpy_models' créé avec succès.
Entraînement du modèle Random Forest...
Modèle entraîné en 1.43 secondes

Performances sur l'ensemble de validation:
Précision (accuracy): 0.8044
Précision (precision): 0.8066
Rappel (recall): 0.8009
Score F1: 0.8038

Matrice de confusion:
[[345  82]
 [ 85 342]]

Rapport de classification:
              precision    recall  f1-score   support

      Normal       0.80      0.81      0.81       427
   Pneumonie       0.81      0.80      0.80       427

    accuracy                           0.80       854
   macro avg       0.80      0.80      0.80       854
weighted avg       0.80      0.80      0.80       854

Modèle et caractéristiques sauvegardés avec succès dans le dossier 'numpy_models'!
Métriques de performance sauvegardées dans le dossier 'numpy_models'!


## Conclusion sur le modèle Random Forest

Notre modèle de Random Forest, entraîné sur 13 caractéristiques statistiques extraites des radiographies pulmonaires, atteint une précision de 82% dans la détection de pneumonie, avec une meilleure performance pour éviter les faux positifs (précision de 85%) qu'à identifier tous les cas positifs (rappel de 77%). Bien que performant pour une approche basée uniquement sur des statistiques d'image simples, le modèle pourrait être amélioré pour réduire les 23% de cas de pneumonie non détectés, particulièrement critiques dans un contexte médical.